In [4491]:
import pandas as pd
import ast
import numpy as np

In [4492]:
nrel = pd.read_csv("dSTAC_US.csv", dtype={"state_fips": "object", "county_fips": "object"})
nrel.head()

,state_abbr,state_fips,county_fips,sector_abbr,energy_value_us_dollars_per_kwh,percent_customers_with_nonzero_sys_size,avg_roof_sqft_customers_with_nonzero_sys_size,avg_roof_sqft_total,hourly_capacity_factor,hourly_capacity_factor_scalar
0,AL,1,1,com,0.0911,0.99,2473.0,4196.0,"[0,0,0,0,0,0,0,799,4190,6338,7315,7366,7049,65...",10000
1,AL,1,1,ind,0.1143,1.00,4196.0,4196.0,"[0,0,0,0,0,0,0,799,4190,6338,7315,7366,7049,65...",10000
2,AL,1,1,res,0.1037,0.54,415.0,349.0,"[0,0,0,0,0,0,0,799,4190,6338,7315,7366,7049,65...",10000
3,AL,1,3,com,0.0743,0.56,2596.0,3973.0,"[0,0,0,0,0,0,0,674,1976,2935,3579,1372,4828,55...",10000
4,AL,1,3,ind,0.0715,0.34,6109.0,3973.0,"[0,0,0,0,0,0,0,674,1976,2935,3579,1372,4828,55...",10000


In [4493]:
nrel = nrel.rename(columns={
    "state_abbr": "State Abbreviation",
    "state_fips": "State FIPS",
    "county_fips": "County FIPS",
    "sector_abbr": "Sector Abbreviation",
    "energy_value_us_dollars_per_kwh": "Energy Value US Dollars Per kWh",
    "percent_customers_with_nonzero_sys_size": "Percent Customers With Nonzero System Size",
    "avg_roof_sqft_customers_with_nonzero_sys_size": "Average Roof Sqft Customers With Nonzero System Size",
    "avg_roof_sqft_total": "Average Root Sqft Total",
    "hourly_capacity_factor": "Hourly Capacity Factor",
    "hourly_capacity_factor_scalar": "Hourly Capacity Factor Scalar"
})

In [4494]:
nrel["State FIPS"] = nrel["State FIPS"].apply(lambda x: x.zfill(2))
nrel["County FIPS"] = nrel["County FIPS"].apply(lambda x: x.zfill(3))

In [4495]:
full_fips = {
    "04013": "Maricopa, AZ",
    "06037": "Los Angeles, CA",
    "12025": "Dade, FL",
    "36047": "Kings, NY",
    "53033": "King, WA",
    "17031": "Cook, IL",
    "32003": "Clark, NV",
    "48201": "Harris, TX",
    "55025": "Dane, WI",
    "51059": "Fairfax, VA"
}

In [4496]:
nrel["Full FIPS"] = nrel["State FIPS"] + nrel["County FIPS"]
nrel["Full FIPS"]

0       01001
1       01001
2       01001
3       01003
4       01003
        ...  
9309    56043
9310    56043
9311    56045
9312    56045
9313    56045
Name: Full FIPS, Length: 9314, dtype: object

In [4497]:
nrel = nrel[nrel["Full FIPS"].isin(full_fips.keys())]

In [4498]:
nrel["County"] = nrel["Full FIPS"].apply(lambda x: full_fips[x])
nrel["County"].sample(5)

8391        Fairfax, VA
529     Los Angeles, CA
8392        Fairfax, VA
2027           Cook, IL
8901           Dane, WI
Name: County, dtype: object

In [4499]:
# Divide hourly_capacity_factor by hourly_capacity_value_scalar to render generation in the correct unit (kWh/kW)
nrel["Real Generation"] = nrel.apply(lambda row: [hour / row["Hourly Capacity Factor Scalar"] for hour in ast.literal_eval(row["Hourly Capacity Factor"])], axis=1)
nrel.head()

,State Abbreviation,State FIPS,County FIPS,Sector Abbreviation,Energy Value US Dollars Per kWh,Percent Customers With Nonzero System Size,Average Roof Sqft Customers With Nonzero System Size,Average Root Sqft Total,Hourly Capacity Factor,Hourly Capacity Factor Scalar,Full FIPS,County,Real Generation
449,AZ,04,013,com,0.0638,1.00,1880.0,1880.0,"[0,0,0,0,0,0,0,264,2843,5097,6196,7248,7482,73...",10000,04013,"Maricopa, AZ","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0264, 0...."
450,AZ,04,013,ind,0.0748,1.00,1906.0,1906.0,"[0,0,0,0,0,0,0,264,2843,5097,6196,7248,7482,73...",10000,04013,"Maricopa, AZ","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0264, 0...."
451,AZ,04,013,res,0.0965,1.00,392.0,392.0,"[0,0,0,0,0,0,0,264,2843,5097,6196,7248,7482,73...",10000,04013,"Maricopa, AZ","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0264, 0...."
527,CA,06,037,com,0.0957,0.91,6141.0,6107.0,"[0,0,0,0,0,0,0,1157,3799,5654,7018,7541,6990,6...",10000,06037,"Los Angeles, CA","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1157, 0...."
528,CA,06,037,ind,0.0685,1.00,10828.0,10828.0,"[0,0,0,0,0,0,0,1157,3799,5654,7018,7541,6990,6...",10000,06037,"Los Angeles, CA","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1157, 0...."


In [4500]:
def agg_func(x):
    # Transpose the lists so that matching indices of the lists are grouped
    # [[1, 2, 3], [4, 5, 6], [7, 8, 9]] -> [[1, 4, 7], [2, 5, 8], [3, 6, 9]]
    transposed = zip(*x.values)

    # Calculate the average of each group of indices
    return [sum(values) / len(values) for values in transposed]

nrel = nrel.groupby("State Abbreviation")["Real Generation"].agg(agg_func)
print(nrel)


State Abbreviation
AZ    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.02639999...
CA    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.11569999...
IL    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0516, 0....
NV    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05629999...
NY    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.015...
TX    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0195, 0....
VA    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.014...
WA    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.167...
WI    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.01200000...
Name: Real Generation, dtype: object


In [ ]:
power_density = 160   # 160 W/m^2 (Rooftop Solar Photovoltaic Technical Potential in the United States: A Detailed Assessment)

# Convert power_density from W/m^2 to kW/m^2
power_density = power_density / 1000   # 1000 W = 1 kW

nrel = nrel.reset_index(drop=False)

# Convert hourly_capacity_factor (kWh) to kWh/m^2
nrel["Energy Per Area"] = nrel.apply(lambda row: [hour * power_density for hour in row["Real Generation"]], axis=1)   # kWh/kW * kW/m^2 = kWh/m^2
nrel[["State Abbreviation", "Energy Per Area"]]

,State Abbreviation,Energy Per Area
0,AZ,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.02639999..."
1,CA,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.11569999..."
2,IL,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0516, 0...."
3,NV,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05629999..."
4,NY,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.015..."
5,TX,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0195, 0...."
6,VA,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.014..."
7,WA,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.167..."
8,WI,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.01200000..."


In [4502]:
for state_abbr in nrel["State Abbreviation"].unique():
    assert len(nrel["Energy Per Area"].values[0]) == 8760

Clean the Visual Crossing data

In [4503]:
vc = pd.read_csv("hourly ten counties 2023-04-01 to 2025-04-09.csv")
vc.head()

,name,datetime,temp,feelslike,dew,humidity,precip,precipprob,preciptype,snow,...,sealevelpressure,cloudcover,visibility,solarradiation,solarenergy,uvindex,severerisk,conditions,icon,stations
0,"Maricopa County, AZ",2023-04-01T00:00:00,51.5,51.5,29.4,42.42,0.0,0.0,NaN,0.0,...,1019.3,5.7,8.5,0.0,0.0,0,10,Clear,clear-night,"KLUF,72278023183,KPHX,KGXF,KBXK,72278523111"
1,"Maricopa County, AZ",2023-04-01T01:00:00,52.3,52.3,30.0,42.20,0.0,0.0,NaN,0.0,...,1018.8,5.7,9.9,0.0,0.0,0,10,Clear,clear-night,"KLUF,72278023183,KPHX,KGXF,KBXK,72278523111"
2,"Maricopa County, AZ",2023-04-01T02:00:00,50.5,50.5,30.0,45.05,0.0,0.0,NaN,0.0,...,1018.6,3.2,9.7,0.0,0.0,0,10,Clear,clear-night,"KLUF,72278023183,KPHX,KGXF,KBXK,72278523111"
3,"Maricopa County, AZ",2023-04-01T03:00:00,46.5,42.9,30.1,52.67,0.0,0.0,NaN,0.0,...,1018.2,3.2,9.8,0.0,0.0,0,10,Clear,clear-night,"KLUF,72278023183,KPHX,KGXF,KBXK,72278523111"
4,"Maricopa County, AZ",2023-04-01T04:00:00,45.9,42.5,30.1,53.86,0.0,0.0,NaN,0.0,...,1017.7,3.2,9.7,0.0,0.0,0,10,Clear,clear-night,"KLUF,72278023183,KPHX,KGXF,KBXK,72278523111"


In [4504]:
vc.columns.values

array(['name', 'datetime', 'temp', 'feelslike', 'dew', 'humidity',
       'precip', 'precipprob', 'preciptype', 'snow', 'snowdepth',
       'windgust', 'windspeed', 'winddir', 'sealevelpressure',
       'cloudcover', 'visibility', 'solarradiation', 'solarenergy',
       'uvindex', 'severerisk', 'conditions', 'icon', 'stations'],
      dtype=object)

In [4505]:
vc = vc.rename(columns={
    "name": "County",
    "datetime": "Datetime",
    "temp": "Temperature",
    "feelslike": "Feels Like",
    "dew": "Dew",
    "humidity": "Humidity",
    "precip": "Precipitation",
    "precipprob": "Precipitation Probability",
    "preciptype": "Precipitation Type",
    "snow": "Snow",
    "snowdepth": "Snow Depth",
    "windgust": "Wind Gust",
    "windspeed": "Wind Speed",
    "winddir": "Wind Direction",
    "sealevelpressure": "Sea Level Pressure",
    "cloudcover": "Cloud Cover",
    "visibility": "Visibility",
    "solarradiation": "Solar Radiation",
    "solarenergy": "Solar Energy",
    "uvindex": "UV Index",
    "severerisk": "Severe Risk",
    "conditions": "Conditions",
    "icon": "Icon",
    "stations": "Stations"})

In [4506]:
vc["Datetime"].dtype

dtype('O')

In [4507]:
vc["Datetime"] = pd.to_datetime(vc["Datetime"])

In [4508]:
print(vc["Datetime"].min())
print(vc["Datetime"].max())

2023-04-01 00:00:00
2025-04-08 23:00:00


In [4509]:
vc = vc[vc["Datetime"].dt.year == 2024]

In [4510]:
vc["County"].unique()

array(['Maricopa County, AZ', 'los angeles, Ca', 'Miami-Dade, FL',
       'Kings County, NY', 'King County, WA', 'Cook county, IL',
       'Clark county, NV', 'Harris County, TX', 'Dane County, WI',
       'Fairfax County, VA'], dtype=object)

In [4511]:
county_replacement = {
    "Maricopa County, AZ": "Maricopa, AZ",
    "los angeles, Ca": "Los Angeles, CA",
    "Miami-Dade, FL": "Dade, FL",
    "Kings County, NY": "Kings, NY",
    "King County, WA": "King, WA",
    "Cook county, IL": "Cook, IL",
    "Clark county, NV": "Clark, NV",
    "Harris County, TX": "Harris, TX",
    "Dane County, WI": "Dane, WI",
    "Fairfax County, VA": "Fairfax, VA"
}
vc["County"] = vc["County"].replace(county_replacement)

In [4512]:
vc["State Abbreviation"] = vc["County"].apply(lambda x: x[-2:])

In [4513]:
vc = vc[vc["State Abbreviation"] == "WI"]
vc.head()

,County,Datetime,Temperature,Feels Like,Dew,Humidity,Precipitation,Precipitation Probability,Precipitation Type,Snow,...,Cloud Cover,Visibility,Solar Radiation,Solar Energy,UV Index,Severe Risk,Conditions,Icon,Stations,State Abbreviation
148489,"Dane, WI",2024-01-01 00:00:00,29.2,22.1,25.1,84.45,0.0,0.0,"rain,snow",0.2,...,100.0,9.9,0.0,0.0,0,10,Overcast,cloudy,"F3620,72641014837,KEFT,KC29,99999900236,KMSN",WI
148490,"Dane, WI",2024-01-01 01:00:00,29.1,22.0,24.4,82.66,0.0,0.0,NaN,0.0,...,100.0,9.9,0.0,0.0,0,10,Overcast,cloudy,"F3620,72641014837,KC29,99999900236,KMSN",WI
148491,"Dane, WI",2024-01-01 02:00:00,29.1,23.9,24.8,83.77,0.0,0.0,NaN,0.0,...,100.0,9.9,0.0,0.0,0,10,Overcast,cloudy,"F3620,72641014837,KC29,99999900236,KMSN",WI
148492,"Dane, WI",2024-01-01 03:00:00,28.9,21.4,24.2,82.33,0.0,0.0,NaN,0.0,...,100.0,9.9,0.0,0.0,0,10,Overcast,cloudy,"F3620,72641014837,KC29,99999900236,KMSN",WI
148493,"Dane, WI",2024-01-01 04:00:00,28.2,20.9,23.7,83.22,0.0,0.0,NaN,0.0,...,100.0,9.9,0.0,0.0,0,10,Overcast,cloudy,"F3620,72641014837,KC29,99999900236,KMSN",WI


In [4514]:
len(vc)

8784

In [4515]:
vc["Datetime"].nunique()

8783

In [4516]:
vc["Datetime"].value_counts()

Datetime
2024-11-03 01:00:00    2
2024-12-31 08:00:00    1
2024-12-31 07:00:00    1
2024-01-01 15:00:00    1
2024-12-31 23:00:00    1
                      ..
2024-12-31 03:00:00    1
2024-12-31 04:00:00    1
2024-12-31 05:00:00    1
2024-12-31 06:00:00    1
2024-12-30 16:00:00    1
Name: count, Length: 8783, dtype: int64

In [4517]:
vc["Datetime"].dt.tz

In [4518]:
vc[(vc["Datetime"].dt.year == 2024) & (vc["Datetime"].dt.month == 11) & (vc["Datetime"].dt.day == 3) & (vc["Datetime"].dt.hour == 1)]

,County,Datetime,Temperature,Feels Like,Dew,Humidity,Precipitation,Precipitation Probability,Precipitation Type,Snow,...,Cloud Cover,Visibility,Solar Radiation,Solar Energy,UV Index,Severe Risk,Conditions,Icon,Stations,State Abbreviation
155857,"Dane, WI",2024-11-03 01:00:00,48.2,45.3,40.6,74.89,0.0,0.0,NaN,0.0,...,100.0,9.9,0.0,0.0,0,10,Overcast,cloudy,"F3620,72641014837,KC29,99999900236,KMSN",WI
155858,"Dane, WI",2024-11-03 01:00:00,49.1,45.4,41.1,73.72,0.0,0.0,NaN,0.0,...,100.0,9.9,0.0,0.0,0,10,Overcast,cloudy,"F3620,72641014837,KC29,99999900236,KMSN",WI


In [4519]:
print(vc["Datetime"].min())
print(vc["Datetime"].max())

2024-01-01 00:00:00
2024-12-31 23:00:00


In [4520]:
vc[vc["Datetime"].dt.floor("h").duplicated(keep=False)]

,County,Datetime,Temperature,Feels Like,Dew,Humidity,Precipitation,Precipitation Probability,Precipitation Type,Snow,...,Cloud Cover,Visibility,Solar Radiation,Solar Energy,UV Index,Severe Risk,Conditions,Icon,Stations,State Abbreviation
155857,"Dane, WI",2024-11-03 01:00:00,48.2,45.3,40.6,74.89,0.0,0.0,NaN,0.0,...,100.0,9.9,0.0,0.0,0,10,Overcast,cloudy,"F3620,72641014837,KC29,99999900236,KMSN",WI
155858,"Dane, WI",2024-11-03 01:00:00,49.1,45.4,41.1,73.72,0.0,0.0,NaN,0.0,...,100.0,9.9,0.0,0.0,0,10,Overcast,cloudy,"F3620,72641014837,KC29,99999900236,KMSN",WI


In [4521]:
vc[vc["Datetime"].dt.second != 0]

,County,Datetime,Temperature,Feels Like,Dew,Humidity,Precipitation,Precipitation Probability,Precipitation Type,Snow,...,Cloud Cover,Visibility,Solar Radiation,Solar Energy,UV Index,Severe Risk,Conditions,Icon,Stations,State Abbreviation


In [4522]:
vc[vc["Datetime"].dt.month == 2]

,County,Datetime,Temperature,Feels Like,Dew,Humidity,Precipitation,Precipitation Probability,Precipitation Type,Snow,...,Cloud Cover,Visibility,Solar Radiation,Solar Energy,UV Index,Severe Risk,Conditions,Icon,Stations,State Abbreviation
149233,"Dane, WI",2024-02-01 00:00:00,39.1,36.7,36.5,90.17,0.0,0.0,NaN,0.0,...,0.0,9.9,0.0,0.0,0,10,Clear,clear-night,"F3620,72641014837,KC29,99999900236,KMSN",WI
149234,"Dane, WI",2024-02-01 01:00:00,38.8,33.8,36.5,91.13,0.0,0.0,NaN,0.0,...,0.0,9.9,0.0,0.0,0,10,Clear,clear-night,"F3620,72641014837,KC29,99999900236,KMSN",WI
149235,"Dane, WI",2024-02-01 02:00:00,36.6,33.8,35.0,93.95,0.0,0.0,NaN,0.0,...,0.0,9.9,0.0,0.0,0,10,Clear,clear-night,"F3620,72641014837,KC29,99999900236,KMSN",WI
149236,"Dane, WI",2024-02-01 03:00:00,33.2,33.2,32.2,96.03,0.0,0.0,NaN,0.0,...,0.0,9.9,0.0,0.0,0,10,Clear,clear-night,"F3620,72641014837,KC29,99999900236,KMSN",WI
149237,"Dane, WI",2024-02-01 04:00:00,34.2,30.6,33.8,98.27,0.0,0.0,NaN,0.0,...,0.0,9.3,0.0,0.0,0,10,Clear,clear-night,"F3620,72641014837,KC29,99999900236,KMSN",WI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149924,"Dane, WI",2024-02-29 19:00:00,41.5,35.1,20.0,41.80,0.0,0.0,NaN,0.0,...,0.0,9.9,0.0,0.0,0,10,Clear,clear-night,"F3620,72641014837,KC29,99999900236,KMSN",WI
149925,"Dane, WI",2024-02-29 20:00:00,39.7,32.2,20.0,44.80,0.0,0.0,NaN,0.0,...,0.0,9.9,0.0,0.0,0,10,Clear,clear-night,"F3620,72641014837,KC29,99999900236,KMSN",WI
149926,"Dane, WI",2024-02-29 21:00:00,39.3,32.3,19.8,45.27,0.0,0.0,NaN,0.0,...,0.0,9.9,0.0,0.0,0,10,Clear,clear-night,"F3620,72641014837,KC29,99999900236,KMSN",WI
149927,"Dane, WI",2024-02-29 22:00:00,38.5,32.5,19.8,46.54,0.0,0.0,NaN,0.0,...,0.0,9.9,0.0,0.0,0,10,Clear,clear-night,"F3620,72641014837,KC29,99999900236,KMSN",WI


2024 was a leap year. February had 29 days. The total hours in 2024 was 8784 instead of 8760. 

In [4523]:
vc = vc[~((vc["Datetime"].dt.month == 2) & (vc["Datetime"].dt.day == 29))]
assert vc[(vc["Datetime"].dt.month == 2) & (vc["Datetime"].dt.day == 29)].empty
assert len(vc) == 8760

In [4524]:
vc = vc.sort_values(by="Datetime")

solarenergy is measured in MJ/(m^2).

In [4525]:
# Convert solarenergy to from MJ/m^2 to kWh/m^2
# 1 kWh = 3.6 MJ
# vc["Solar Energy"] = vc["Solar Energy"].apply(lambda x: x / 3.6)
vc["Solar Energy"] = vc["Solar Energy"] / 3.6

Calculate R^2

In [4526]:
# Calculate the mean of the actual values
mean_actual = vc["Solar Energy"].mean()
print(mean_actual)

0.10996004566210045


In [4527]:
# Calculate the total sum of squares
# tss = ((vc["Solar Energy"] - mean_actual) ** 2).sum()
tss = np.sum((vc["Solar Energy"] - mean_actual) ** 2)
print(tss)

314.6249674974632


In [4528]:
nrel = nrel.explode("Energy Per Area")
nrel.sample(5)

,State Abbreviation,Real Generation,Energy Per Area
8,WI,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.01200000...",0.6988
2,IL,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0516, 0....",0.0
7,WA,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.167...",0.7645
2,IL,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0516, 0....",0.0847
2,IL,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0516, 0....",0.0


In [4529]:
nrel.drop(["Real Generation"], axis="columns", inplace=True)
nrel.sample(5)

,State Abbreviation,Energy Per Area
0,AZ,0.5439
7,WA,0.0
3,NV,0.0
7,WA,0.0
5,TX,0.0001


In [4530]:
nrel["Energy Per Area"] = nrel["Energy Per Area"].astype("float64")

In [4531]:
nrel = nrel[nrel["State Abbreviation"] == "WI"]
len(nrel)

8760

In [4532]:
print(nrel["Energy Per Area"].mean())

0.15492762557077625


In [4533]:
print(vc["Solar Energy"].dtype)
print(nrel["Energy Per Area"].dtype)

float64
float64


In [4534]:
assert vc["Solar Energy"].isna().sum() == 0
assert nrel["Energy Per Area"].isna().sum() == 0
assert len(vc) == len(nrel)
assert vc["Solar Energy"].dtype == nrel["Energy Per Area"].dtype

In [4535]:
# Calculate the residual sum of squares
# nrel["Residuals"] = vc["Solar Energy"] - nrel["Energy Per Area"]
# nrel.sample(10)
# print(type(vc))
# print(type(vc["Solar Energy"]))
# print(len(vc))
# print(vc.index.duplicated())
# print(vc.index)
# print(type(nrel))
# print(type(nrel["Energy Per Area"]))
# print(len(nrel))
# print(nrel.index.duplicated())
# print(nrel.index)
print(vc["Solar Energy"].iloc[515])
print(nrel["Energy Per Area"].iloc[515])
nrel = nrel.reset_index(drop=True)
vc = vc.reset_index(drop=True)
nrel["Residuals"] = vc["Solar Energy"] - nrel["Energy Per Area"]
rss = np.sum((nrel["Residuals"]) ** 2)
print(rss)

0.16666666666666666
0.0597
293.97781456592594


Calculate R squared

In [4536]:
r2 = 1 - rss / tss
print(r2)

0.06562464859596295
